In [1]:
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import nlp_helpers as nh

In [2]:
# Load the data, and vocab
    
data_path = '../input/preprocessing-with-stem/preprocessed_data_with_stem.obj'
# vocab_path = os.path.join(root_path, 'vocab_no_stem.obj')
# labels_path = os.path.join(root_path, 'labels_to_int.obj')

In [3]:
data = nh.load_pickle_file(data_path)

In [4]:
data.head()

,id,text,dialect
0,1175358310087892992,"[نهي, نفض, يغر]",IQ
1,1175416117793349632,"[يعن, بشر, وحش, طلب, غرب, حرم, يءم, بدن, ينع, ...",IQ
2,1175450108898565888,"[كلم, خلج]",IQ
3,1175471073770573824,"[لمل, رور, ورح, حلوه]",IQ
4,1175496913145217024,"[وين, غيب, حمد]",IQ


In [5]:
len(data)

458197

In [6]:
data['tweet_length'] = [len(tweet) for tweet in data['text']]

In [7]:
threshold = 2
data = data[data['tweet_length'] > threshold]

In [8]:
data.reset_index(inplace=True, drop=True)

In [9]:
len(data)

445111

### Convert each tweet to one string
Each tweet is a list of words, we want to return it back as one string separated by spaces

In [10]:
data_string = data['text'].apply(nh.get_one_string)

In [11]:
data_string.head()

0                                          نهي نفض يغر
1              يعن بشر وحش طلب غرب حرم يءم بدن ينع رهب
2                                     لمل رور ورح حلوه
3                                          وين غيب حمد
4    ياخ رهب عرق سعد لسط وين شكل باب فرضخليجي اعند ...
Name: text, dtype: object

In [12]:
# labels_to_int: dictionary, where its keys are the labels,
# and its values are integers starting from 0
labels = data['dialect'].unique()
labels_to_int, int_to_labels = nh.get_mappings(labels, i=0)

In [13]:
# save labels_to_int, and int_to_labels, they will be needed in inference
nh.save_pickle_file(labels_to_int, 'labels_to_int.obj')
nh.save_pickle_file(int_to_labels, 'int_to_labels.obj')

In [14]:
split_frac = 0.2

## split data into training, validation, and test data (features and labels, x and y)
X = data_string.values
y = data['dialect'].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=split_frac, shuffle=True, 
                                                  random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, shuffle=True, 
                                                random_state=42, stratify=y_val)
## print out the shapes of your resultant feature data
print('\t\t\tFeatures Shapes:')
print('Train set: \t\t{}'.format(X_train.shape),
     '\nValidation set: \t{}'.format(X_val.shape),
     '\nTest set: \t\t{}'.format(X_test.shape))

			Features Shapes:
Train set: 		(356088,) 
Validation set: 	(44511,) 
Test set: 		(44512,)


In [15]:
# setting ngram_range to (1, 2) to count individual words and bigram words (sequence of two words)
tfidf = TfidfVectorizer(ngram_range=(1, 2), dtype=np.float64)
classifier = MultinomialNB()

pipe = Pipeline([('vectorizer', tfidf), ('classifier', classifier)])
pipe.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer(ngram_range=(1, 2))),
                ('classifier', MultinomialNB())])

In [16]:
y_hat = pipe.predict(X_test)

In [17]:
print('Predicted:\n', y_hat)
print('Actual:\n', y_test)

Predicted:
 ['PL' 'KW' 'EG' ... 'KW' 'KW' 'EG']
Actual:
 ['SY' 'KW' 'PL' ... 'AE' 'SA' 'KW']


In [18]:
from sklearn.metrics import f1_score

f1_score(y_test, y_hat, average='macro')

0.1742671858058694

In [19]:

# pipe2 = Pipeline([('vectorizer', tfidf), ('classifier', SVC())])

# pipe2.fit(X_train, y_train)

In [20]:
y_hat = pipe.predict(X_test)
print(y_hat)
print(y_test)

['PL' 'KW' 'EG' ... 'KW' 'KW' 'EG']
['SY' 'KW' 'PL' ... 'AE' 'SA' 'KW']


In [21]:
y_hat = pipe.predict(X_val)
f1_score(y_val, y_hat, average='macro')

0.17222606419099873

In [22]:
def test(classifier):
    pipe_line = Pipeline([('vectorizer', tfidf), ('classifier', classifier)])
    pipe_line.fit(X_train, y_train)

    y_hat = pipe_line.predict(X_val)

    print('Predicted:\n', y_hat)
    print('Actual:\n', y_val)
    print(f1_score(y_val, y_hat, average='macro'))
    nh.save_pickle_file(pipe_line, 'pipe_rf_20_with_stem.obj')
    return f1_score(y_val, y_hat, average='macro')

In [23]:
rf = RandomForestClassifier(n_estimators=20, random_state=42, class_weight='balanced')
test(rf)

Predicted:
 ['JO' 'PL' 'QA' ... 'PL' 'EG' 'IQ']
Actual:
 ['JO' 'PL' 'YE' ... 'PL' 'JO' 'IQ']
0.3054567468582396


0.3054567468582396

In [24]:
nh.save_pickle_file(rf, 'rf_20.obj')